<a href="https://colab.research.google.com/github/SinthanaPrabakaran/Face-Recognition/blob/main/Face_recogntion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import dlib
import time
import psutil
import statistics
!pip install deepface
from deepface import DeepFace
from collections import deque
import threading
import queue
import logging
import tkinter as tk
from tkinter import messagebox, font
import cv2
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from google.colab.patches import cv2_imshow
from IPython.display import display, HTML, Javascript

class NumberLockSystem:
    def __init__(self, master):
        self.master = master
        self.master.title("Number Lock System")
        self.master.geometry("300x250")
        self.master.configure(bg="#f0f8ff")  # Light blue background

        # Set a predefined lock code
        self.lock_code = "1207"

        self.create_widgets()

    def create_widgets(self):
        # Set font style
        title_font = font.Font(family="Helvetica", size=14, weight="bold")

        self.instruction_label = tk.Label(self.master, text="Enter the lock code:", bg="#f0f8ff", font=title_font)
        self.instruction_label.pack(pady=10)

        self.code_entry = tk.Entry(self.master, show="*", font=("Helvetica", 16), width=10)
        self.code_entry.pack(pady=10)

        self.submit_button = tk.Button(self.master, text="Submit", command=self.check_code, bg="#4CAF50", fg="white", font=("Helvetica", 12))
        self.submit_button.pack(pady=5)

        self.clear_button = tk.Button(self.master, text="Clear", command=self.clear_input, bg="#f44336", fg="white", font=("Helvetica", 12))
        self.clear_button.pack(pady=5)

        # Adding a footer label
        self.footer_label = tk.Label(self.master, text="© 2024 Home Security", bg="#f0f8ff", font=("Helvetica", 8))
        self.footer_label.pack(side="bottom", pady=10)

    def check_code(self):
        input_code = self.code_entry.get()

        if input_code == self.lock_code:
            messagebox.showinfo("Access Granted", "Welcome Home!")
            self.master.destroy()  # Close the entire application
        else:
            messagebox.showerror("Access Denied", "Incorrect code, try again.")
            self.clear_input()

    def clear_input(self):
        self.code_entry.delete(0, tk.END)

class EnhancedDynamicFaceRecognition:
    def __init__(self, reference_img1, reference_img2):
        # Initialize performance metrics
        self.fps_queue = deque(maxlen=30)
        self.processing_times = {
            'haar': deque(maxlen=30),
            'dlib': deque(maxlen=30),
            'deepface': deque(maxlen=30)
        }

        # Reference images for face recognition
        self.reference_img1 = cv2.imread(reference_img1)
        self.reference_img2 = cv2.imread(reference_img2)

        # Initialize logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

        # Initialize different detectors
        self.haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.dlib_detector = dlib.get_frontal_face_detector()
        self.dlib_predictor = dlib.shape_predictor('/shape_predictor_68_face_landmarks .dat')

        # Detection modes with priority and characteristics
        self.detection_modes = {
            'fast': {
                'method': self._detect_faces_haar,
                'accuracy_score': 0.6,
                'computational_cost': 0.2,
                'lighting_sensitivity': 0.3
            },
            'accurate': {
                'method': self._detect_faces_dlib,
                'accuracy_score': 0.8,
                'computational_cost': 0.5,
                'lighting_sensitivity': 0.5
            },
            'deep': {
                'method': self._detect_faces_deepface,
                'accuracy_score': 0.9,
                'computational_cost': 0.8,
                'lighting_sensitivity': 0.7
            }
        }

        self.current_mode = 'fast'
        self.face_match = 0

        # Performance and resource thresholds
        self.performance_metrics = {
            'cpu_threshold_low': 30,
            'cpu_threshold_high': 70,
            'fps_threshold_low': 15,
            'fps_threshold_high': 25,
            'processing_time_threshold': 0.05  # 50ms
        }

        # Initialize processing queue
        self.frame_queue = queue.Queue(maxsize=2)
        self.result_queue = queue.Queue(maxsize=2)

        # Start processing thread
        self.processing_thread = threading.Thread(target=self._process_frames)
        self.processing_thread.daemon = True
        self.processing_thread.start()

    def _process_frames(self):
        """
        Process frames from the queue.

        This is a placeholder method. You need to implement the actual frame processing logic here.
        """
        while True:
            try:
                # Get a frame from the queue
                frame = self.frame_queue.get()

                # Process the frame (e.g., face detection, recognition)
                # ... your frame processing logic here ...

                # Put the result back in the result queue
                self.result_queue.put(...)  # Replace ... with the actual result

            except queue.Empty:
                # Handle empty queue (e.g., wait for a new frame)
                pass

    def _detect_faces_haar(self, frame):
        """
        Detect faces using Haar Cascade classifier.
        """
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.haar_cascade.detectMultiScale(gray, 1.1, 4)
        return faces

    def _detect_faces_dlib(self, frame):
        """
        Detect faces using Dlib's HOG-based face detector.
        """
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.dlib_detector(gray)
        return faces

    def _detect_faces_deepface(self, frame):
        """
        Detect faces using DeepFace.
        """
        try:
            detections = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
            faces = [(detection['region']['x'], detection['region']['y'],
                      detection['region']['w'], detection['region']['h'])
                     for detection in detections]
            return faces
        except ValueError:
            return []  # Return empty list if no faces are detected


    def _check_face(self, frame):
        """
        Check if the frame matches any of the reference images
        """
        try:
            # Reset face match before checking
            self.face_match = 0

            # Check against first reference image
            if DeepFace.verify(frame, self.reference_img1.copy())['verified']:
                self.face_match = 1
                return True

            # Check against second reference image
            elif DeepFace.verify(frame, self.reference_img2.copy())['verified']:
                self.face_match = 2
                return True

        except ValueError:
            self.face_match = 0

        return False

    def _launch_lock_system(self):
        """
        Launch the number lock system using Colab forms and JavaScript.
        """
        # Predefined lock code
        lock_code = "1207"

        # HTML code for the form
        form_html = f"""
        <div id="number-lock">
            <h2>Enter Lock Code</h2>
            <input type="password" id="lock-code-input" placeholder="Enter code">
            <button onclick="checkCode('{lock_code}')">Submit</button>
            <p id="message"></p>
        </div>
        """

        # Use the imported HTML class to display the form
        display(HTML(form_html))

        # JavaScript code for code verification
        js_code = """
        function checkCode(correctCode) {
            const inputCode = document.getElementById('lock-code-input').value;
            const messageElement = document.getElementById('message');

            if (inputCode === correctCode) {
                messageElement.textContent = 'Access Granted! Welcome Home!';
                messageElement.style.color = 'green';
                // You can add further actions here, like stopping the face recognition loop
                google.colab.kernel.invokeFunction('notify_access_granted', [], {});
            } else {
                messageElement.textContent = 'Incorrect code, try again.';
                messageElement.style.color = 'red';
            }
        }
        """
        display(Javascript(js_code))

    def run(self):
        """
        Run the face recognition system.
        """
        counter = 0

        while True:
            start_time = time.time()
            try:
                filename = take_photo()
                frame = cv2.imread(filename)
            except Exception as err:
                print(str(err))
                break

            # Periodically check face recognition (every 30 frames)
            if counter % 30 == 0:
                face_thread = threading.Thread(target=self._check_face, args=(frame.copy(),))
                face_thread.start()

            # Display face match status
            if self.face_match in [1, 2]:
                # Display "MATCH FOUND!" for a short duration
                match_message = "MATCH FOUND!"
                cv2.putText(frame, match_message, (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2_imshow(frame)
                cv2.waitKey(2000)  # Display for 2 seconds
                cv2.destroyAllWindows()

                # Launch number lock system
                self._launch_lock_system()
                break  # Exit the loop after launching the lock system
            else:
                cv2.putText(frame, "NO MATCH!", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Calculate and update FPS
            elapsed_time = time.time() - start_time
            current_fps = 1 / elapsed_time if elapsed_time > 0 else 0
            self.fps_queue.append(current_fps)

            # Display the frame using cv2_imshow
            cv2_imshow(frame)

            counter += 1

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cv2.destroyAllWindows()


if __name__ == "__main__":
    face_recognizer = EnhancedDynamicFaceRecognition(
        reference_img1="/Sinthana.jpg",
        reference_img2="/photo.jpg"
    )
    face_recognizer.run()